In [98]:
import pandas as pd

In [109]:
# Read the data
artistas = pd.read_csv('Artistas.csv', sep=',')
artistas = artistas.drop_duplicates()

entradas = pd.read_csv('Entradas Cortesia.csv', sep=',')
entradas = entradas.drop_duplicates()

eventos = pd.read_csv('Eventos.csv', sep=',')
eventos = eventos.drop_duplicates()

hosp_traslados = pd.read_csv('Hospedajes y Traslados.csv', sep=',')
hosp_traslados = hosp_traslados.drop_duplicates()

productoras = pd.read_csv('Productoras.csv', sep=',')
productoras = productoras.drop_duplicates()

tour = pd.read_csv('Tour.csv', sep=',')
tour = tour.drop_duplicates()


artistas = artistas.rename(columns={'Nombre artistico': 'Artista'})
entradas = entradas.rename(columns={'evento': 'Evento'})
entradas = entradas.rename(columns={'artista': 'Artista'})

In [110]:
artistas['id'] = range(1, len(artistas) + 1)
entradas['id'] = range(1, len(entradas) + 1)
eventos['id'] = range(1, len(eventos) + 1)
hosp_traslados['id'] = range(1, len(hosp_traslados) + 1)
productoras['id'] = range(1, len(productoras) + 1)
tour['id'] = range(1, len(tour) + 1)

In [111]:
entradas = entradas.merge(artistas[['id', 'Artista']], on='Artista', how='left')
entradas = entradas.rename(columns={'id_x': 'id_entrada', 'id_y': 'id_artista'})
eventos = eventos.merge(artistas[['id', 'Artista']], on='Artista', how='left')
eventos = eventos.rename(columns={'id_x': 'id_evento', 'id_y': 'id_artista'})
entradas = entradas.drop(columns=['Artista'])
eventos = eventos.drop(columns=['Artista'])
entradas = entradas.merge(eventos[['id_evento', 'Evento']], on='Evento', how='left')
entradas = entradas.drop(columns=['Evento'])

In [112]:
entradas = entradas.rename(columns={'id_entrada': 'ID_entrada', 'id_artista': 'ID_artista', 'id_evento': 'ID_evento'})
artistas = artistas.rename(columns={'Artista': 'Nombre', 'id': 'ID_artista', 'Inicio carrera': 'Fecha_inicio', 'Nº contacto': 'Contacto'})
# export artistas to csv
artistas.to_csv('artistas_bdd.csv', index=False)
# export entradas to csv
entradas.to_csv('entradas_bdd.csv', index=False)

In [113]:
artistas

,Nombre,Fecha_inicio,Contacto,ID_artista
0,Duki,14/02/2019,947967523,1
1,Backstreet Boys,23/01/1994,933555053,2
2,Michael Jackson,04/02/1980,973673018,3
3,Mon Laferte,18/07/2008,982606150,4
4,Khea,16/06/2013,927484546,5
...,...,...,...,...
374,La Oreja de Van Gogh,08/03/2001,966357123,196
378,Bizarrap,11/12/2017,957841287,197
397,Nicki Minaj,02/07/2010,975557868,198
411,Camila,26/06/2009,965638977,199
